# Analiza i Razvoj Modela za Klasifikaciju Proizvoda

Ovaj notebook sadrži kompletan proces:
1. Učitavanje i istraživanje podataka (EDA)
2. Pripremu i čišćenje podataka
3. Inženjering karakteristika (TF-IDF)
4. Poređenje modela i evaluaciju


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import os

# 1. Učitavanje podataka
print("Učitavanje podataka...")
df = pd.read_csv('products.csv')

# Osnovne informacije
print(df.info())
print("\nPrvih 5 redova:")
print(df.head())

# Provera nedostajućih vrednosti
print("\nNedostajuće vrednosti:")
print(df.isnull().sum())

# Čišćenje podataka - uklanjanje redova bez kategorije ili naslova
df = df.dropna(subset=['Product Title', ' Category Label'])

# 2. Exploratory Data Analysis (EDA)
print("\nBroj proizvoda po kategorijama:")
category_counts = df[' Category Label'].value_counts()
print(category_counts)

# 3. Preprocessing & Feature Engineering
# Za početak koristimo samo Product Title
X = df['Product Title']
y = df[' Category Label']

# Podela na trening i test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# 4. Model Selection & Training
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
}

for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    print(f"\nModel: {name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    # print(classification_report(y_test, y_pred))

# 5. Finalni model (Logistic Regression se obično dobro pokazuje za tekst)
final_model = LogisticRegression(max_iter=1000)
final_model.fit(X_train_tfidf, y_train)

# Čuvanje modela i vektoraizera
joblib.dump(final_model, 'model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
print("\nModel i vektoraizer su sačuvani.")
